# ASHRAE - Great Energy Predictor III


SUMMARY: `test.csv` is big data and takes time to load. I will convert competition data to feather format for fast `pandas.DataFrame` loading!

In [1]:
import os
import random
import gc

import tqdm
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import preprocessing
from sklearn.model_selection import KFold
import lightgbm as lgb
import xgboost as xgb
import catboost as cb

In [2]:
# Copy from https://www.kaggle.com/gemartin/load-data-reduce-memory-usage by @gemartin
# Modified to support timestamp type
# Modified to add option to use float16 or not. feather format does not support float16.
from pandas.api.types import is_datetime64_any_dtype as is_datetime

def reduce_mem_usage(df, use_float16=False):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        if is_datetime(df[col]):
            # skip datetime type
            continue
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if use_float16 and c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df


def import_data(file):
    """create a dataframe and optimize its memory usage"""
    df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
    df = reduce_mem_usage(df)
    return df

In [11]:
%%time

# Read data...
#root = '../input/ashrae-energy-prediction'
root = os.path.join('..', 'input', 'ashrae-energy-prediction')

train_df = pd.read_csv(os.path.join(root, 'train.csv'))
weather_train_df = pd.read_csv(os.path.join(root, 'weather_train.csv'))
test_df = pd.read_csv(os.path.join(root, 'test.csv'))
weather_test_df = pd.read_csv(os.path.join(root, 'weather_test.csv'))
building_meta_df = pd.read_csv(os.path.join(root, 'building_metadata.csv'))
sample_submission = pd.read_csv(os.path.join(root, 'sample_submission.csv'))

Wall time: 30 s


In [12]:
train_df['timestamp'] = pd.to_datetime(train_df['timestamp'])
test_df['timestamp'] = pd.to_datetime(test_df['timestamp'])
weather_train_df['timestamp'] = pd.to_datetime(weather_train_df['timestamp'])
weather_test_df['timestamp'] = pd.to_datetime(weather_test_df['timestamp'])

In [13]:
# # categorize primary_use column to reduce memory on merge...

# primary_use_dict = {key: value for value, key in enumerate(primary_use_list)} 
# print('primary_use_dict: ', primary_use_dict)
# building_meta_df['primary_use'] = building_meta_df['primary_use'].map(primary_use_dict)

# gc.collect()

In [14]:
reduce_mem_usage(train_df)
reduce_mem_usage(test_df)
reduce_mem_usage(building_meta_df)
reduce_mem_usage(weather_train_df)
reduce_mem_usage(weather_test_df)

Memory usage of dataframe is 616.95 MB
Memory usage after optimization is: 289.19 MB
Decreased by 53.1%
Memory usage of dataframe is 1272.51 MB
Memory usage after optimization is: 596.49 MB
Decreased by 53.1%
Memory usage of dataframe is 0.07 MB
Memory usage after optimization is: 0.02 MB
Decreased by 65.4%
Memory usage of dataframe is 9.60 MB
Memory usage after optimization is: 4.93 MB
Decreased by 48.6%
Memory usage of dataframe is 19.04 MB
Memory usage after optimization is: 9.78 MB
Decreased by 48.6%


,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,2017-01-01 00:00:00,17.799999,4.0,11.7,NaN,1021.400024,100.0,3.6
1,0,2017-01-01 01:00:00,17.799999,2.0,12.8,0.0,1022.000000,130.0,3.1
2,0,2017-01-01 02:00:00,16.100000,0.0,12.8,0.0,1021.900024,140.0,3.1
3,0,2017-01-01 03:00:00,17.200001,0.0,13.3,0.0,1022.200012,140.0,3.1
4,0,2017-01-01 04:00:00,16.700001,2.0,13.3,0.0,1022.299988,130.0,2.6
...,...,...,...,...,...,...,...,...,...
277238,15,2018-12-31 19:00:00,3.300000,NaN,1.7,NaN,1018.299988,150.0,7.7
277239,15,2018-12-31 20:00:00,2.800000,NaN,1.1,NaN,1017.799988,140.0,5.1
277240,15,2018-12-31 21:00:00,2.800000,NaN,1.7,-1.0,1017.200012,140.0,6.2
277241,15,2018-12-31 22:00:00,2.800000,NaN,2.2,8.0,1016.099976,140.0,5.1


# Save data in feather format

In [16]:
%%time

output_path = os.path.join('..', 'output', 'ashrae-feather-format-for-fast-loading')

train_df.to_feather(os.path.join(output_path, 'train.feather'))
test_df.to_feather(os.path.join(output_path, 'test.feather'))
weather_train_df.to_feather(os.path.join(output_path, 'weather_train.feather'))
weather_test_df.to_feather(os.path.join(output_path, 'weather_test.feather'))
building_meta_df.to_feather(os.path.join(output_path, 'building_metadata.feather'))
sample_submission.to_feather(os.path.join(output_path, 'sample_submission.feather'))

Wall time: 9.54 s


# Read data in feather format

You can see "+ Add data" button on top-right of notebook, press this button and add output of this kernel, then you can use above saved feather data frame for fast loading!

Let's see how fast it is.

In [17]:
%%time

train_df = pd.read_feather(os.path.join(output_path, 'train.feather'))
weather_train_df = pd.read_feather(os.path.join(output_path, 'weather_train.feather'))
test_df = pd.read_feather(os.path.join(output_path, 'test.feather'))
weather_test_df = pd.read_feather(os.path.join(output_path, 'weather_test.feather'))
building_meta_df = pd.read_feather(os.path.join(output_path, 'building_metadata.feather'))
sample_submission = pd.read_feather(os.path.join(output_path, 'sample_submission.feather'))

C:\Users\sbbpan\AppData\Local\Continuum\anaconda3\lib\site-packages\pyarrow\pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


Wall time: 1.05 s


Reduced 37.1 sec to 1.51 sec!! 😄😄😄